In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(color_codes=True,style='darkgrid')

## Problem Statement
Welcome to Sigma Cab Private Limited - a cab aggregator service. Their customers can download their app on smartphones and book a cab from any where in the cities they operate in. They, in turn search for cabs from various service providers and provide the best option to their client across available options. They have been in operation for little less than a year now. During this period, they have captured surgepricingtype from the service providers.

You have been hired by Sigma Cabs as a Data Scientist and have been asked to build a predictive model, which could help them in predicting the surgepricingtype pro-actively. This would in turn help them in matching the right cabs with the right customers quickly and efficiently.



## Data
Variable Definition
TripID - ID for TRIP (Can not be used for purposes of modelling) TripDistance - The distance for the trip requested by the customer
TypeofCab - Category of the cab requested by the customer
CustomerSinceMonths - Customer using cab services since n months; 0 month means current month
LifeStyleIndex - Proprietary index created by Sigma Cabs showing lifestyle of the customer based on their behaviour
ConfidenceLifeStyleIndex - Category showing confidence on the index mentioned above DestinationType - Sigma Cabs divides any destination in one of the 14 categories
CustomerRating - Average of life time ratings of the customer till date CancellationLast1Month - Number of trips cancelled by the customer in last 1 month Var1, Var2 and Var3 - Continuous variables masked by the company. Can be used for modelling purposes Gender - Gender of the customer SurgePricing_Type - Predictor variable can be of 3 types

In [ ]:
file = '/kaggle/input/sigmacabprediction/SigmaCab-Train.csv'
df = pd.read_csv(file)

In [ ]:
df.head()

We dont need to use Trip ID for our modelling, So we drop this column.

In [ ]:
df = df.drop(['Trip_ID'],axis=1)

In [ ]:
df.describe(include='all').transpose()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

There are lot of missing values we should deal with.

In [ ]:
df.hist(figsize=(15,10))

In [ ]:
sns.countplot(df['Surge_Pricing_Type'],hue=df['Gender'])

In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
sns.countplot(df['Type_of_Cab'])

plt.subplot(1,2,2)
sns.countplot(df['Type_of_Cab'],hue=df['Surge_Pricing_Type'])

In [ ]:
sns.countplot(df['Destination_Type'])

In [ ]:
sns.boxplot(df['Life_Style_Index'])

In [ ]:
sns.pointplot(df['Surge_Pricing_Type'],df['Cancellation_Last_1Month'])

In [ ]:
sns.boxplot(df['Surge_Pricing_Type'],df['Customer_Rating'])

In [ ]:
sns.boxplot(df['Surge_Pricing_Type'],df['Var3'])

In [ ]:
df.groupby(['Destination_Type'])['Surge_Pricing_Type'].value_counts()

In [ ]:
df.groupby(['Destination_Type'])['Surge_Pricing_Type'].value_counts().plot()

In [ ]:
df['Destination_Type'].value_counts()

In [ ]:
sns.boxplot(df['Surge_Pricing_Type'],df['Life_Style_Index'])

In [ ]:
sns.countplot(df['Customer_Since_Months'],hue=df['Surge_Pricing_Type'])

In [ ]:
sns.boxplot(df['Surge_Pricing_Type'],df['Trip_Distance'])

In [ ]:
sns.pointplot(df['Surge_Pricing_Type'],df['Trip_Distance'])

In [ ]:
df.groupby(['Destination_Type'])['Trip_Distance'].mean().sort_values(ascending=False).plot()

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot(df['Destination_Type'],df['Trip_Distance'])

In [ ]:
df.groupby(['Type_of_Cab','Surge_Pricing_Type'])['Surge_Pricing_Type'].count()

In [ ]:
corr = df.corr()
plt.figure(figsize=(10,8))
sns.heatmap(corr,annot=True)

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
imputer = SimpleImputer(missing_values=np.nan,strategy='most_frequent')

In [ ]:
imputed_Toc = imputer.fit_transform(np.array(df['Type_of_Cab']).reshape(-1,1))

In [ ]:
Toc = pd.DataFrame(imputed_Toc,columns=['Type_of_Cab'])

In [ ]:
Toc.head()

In [ ]:
Toc.isna().any()

In [ ]:
df['imputed_Toc'] = df['Type_of_Cab'].isna()

In [ ]:
df['imputed_Toc'] = df['imputed_Toc'].replace([True,False],[1,0])

In [ ]:
df.head()

In [ ]:
df = df.drop(['Type_of_Cab'],axis=1)

In [ ]:
df = pd.concat([df,Toc],axis=1)

In [ ]:
df.head()

In [ ]:
df['Customer_Since_Months'].value_counts()

In [ ]:
df['imputed_Csm'] = df['Customer_Since_Months'].isna()

In [ ]:
df['imputed_Csm'] = df['imputed_Csm'].replace([True,False],[1,0])

In [ ]:
df['Customer_Since_Months'] = df['Customer_Since_Months'].fillna(0)

In [ ]:
df['Customer_Since_Months'].value_counts()

In [ ]:
df.isna().sum()

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df['Destination_Type'].value_counts()

In [ ]:
DT = {'A':14,'B':13,'C':12,'D':11,'E':10,'F':9,'G':8,'H':7,'I':6,'J':5,'K':4,'L':3,'N':2,'M':1}

In [ ]:
df['Destination_Type'] = df['Destination_Type'].map(DT)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
T = pd.get_dummies(df['Type_of_Cab'],drop_first=True,prefix='Toc')

In [ ]:
T

In [ ]:
df = pd.concat([df,T],axis=1)

In [ ]:
G = pd.get_dummies(df['Gender'],drop_first=True,prefix='G')

In [ ]:
df = pd.concat([df,G],axis=1)

In [ ]:
df = df.drop(['Type_of_Cab','Gender'],axis=1)

In [ ]:
df['Confidence_Life_Style_Index'] = df['Confidence_Life_Style_Index'].replace(np.nan,'mis')

In [ ]:
C = pd.get_dummies(df['Confidence_Life_Style_Index'],drop_first=True,prefix='CLSI')

In [ ]:
df = pd.concat([df,C],axis=1)
df = df.drop(['Confidence_Life_Style_Index'],axis=1)

In [ ]:
df.head()

In [ ]:
cols = df.columns

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
imp = IterativeImputer(max_iter=10,random_state=0)

In [ ]:
df = imp.fit_transform(df)

In [ ]:
df = pd.DataFrame(df,columns=cols)

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
corr = df.corr()
plt.figure(figsize=(20,10))
sns.heatmap(corr,annot=True)

In [ ]:
df.describe().transpose()

In [ ]:
X = df.drop(['Surge_Pricing_Type'],axis=1)
y = df['Surge_Pricing_Type']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0,stratify=y)

In [ ]:
x_train.shape

In [ ]:
y_test.shape

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
xs_train = scaler.fit_transform(x_train)
xs_test = scaler.fit_transform(x_test)

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
log = LogisticRegression(random_state=1)

In [ ]:
model1 = log.fit(xs_train,y_train)

In [ ]:
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score

In [ ]:
y_pred1 = model1.predict(xs_test)

In [ ]:
accuracy_score(y_test,y_pred1)

In [ ]:
f1_score(y_test,y_pred1,average='macro')

## Support Vector Machines

In [ ]:
from sklearn.svm import LinearSVC,svc

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dtc = DecisionTreeClassifier(max_depth=15,ccp_alpha=0.0)

In [ ]:
model3 = dtc.fit(x_train,y_train)

In [ ]:
y_pred3 = model3.predict(x_test)

In [ ]:
accuracy_score(y_test,y_pred3)

In [ ]:
f1_score(y_test,y_pred3,average='macro')

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier()

In [ ]:
model4 = rfc.fit(x_train,y_train)

In [ ]:
y_pred4 = model4.predict(x_test)

In [ ]:
accuracy_score(y_test,y_pred4)

In [ ]:
f1_score(y_test,y_pred4,average='macro')

In [ ]:
from sklearn.ensemble import AdaBoostClassifier


In [ ]:
abcl = AdaBoostClassifier(n_estimators=300,learning_rate=0.3)

In [ ]:
model5 = abcl.fit(x_train,y_train)

In [ ]:
y_pred5 = model5.predict(x_test)

In [ ]:
accuracy_score(y_test,y_pred5)

In [ ]:
f1_score(y_test,y_pred5,average='macro')

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gbcl  = GradientBoostingClassifier()

In [ ]:
model6 = gbcl.fit(x_train,y_train)

In [ ]:
y_pred6 = model6.predict(x_test)

In [ ]:
accuracy_score(y_test,y_pred6)